In [3]:
pip install tensorflow

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from sklearn.preprocessing import *
from sklearn.model_selection import*
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [ ]:
"""
 Our Fixed Parameters
"""
TRAIN_DIR = 'Train';TEST_DIR = 'Test'
IMG_SIZE =50;LR = 0.001
MODEL_NAME = 'Sports-cnn';epochs=25

In [ ]:
def create_label(image_name):
    """
        [1] Description: Encoded Each Class Between 0:5
        [2] Arguments: image_name
        [3] Return: numpy array of encoded value
    """
    word_label = image_name.split('_')[0]
    if word_label == 'Tennis':#4
        return np.array([0,1,0,0,0,0])
    elif word_label == 'Football':#1
        return np.array([0,0,0,0,1,0])
    elif word_label == 'Yoga':#5
        return np.array([1,0,0,0,0,0])
    elif word_label == 'Basketball':#0
        return np.array([0,0,0,0,0,1])
    elif word_label == 'Swimming':#3
        return np.array([0,0,1,0,0,0])
    elif word_label == 'Rowing':#2
        return np.array([0,0,0,1,0,0])

In [ ]:
def create_train_data():
    """
        [1] Description: Create Train Data (Value of Image,Label)
        [2] Arguments: None
        [3] Return: training_data (List)
    """
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        path = os.path.join(TRAIN_DIR, img)
        img_data = cv2.imread('data\Train',0)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img_data), create_label(img)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data


In [ ]:
os.listdir(TRAIN_DIR)

In [ ]:
labels=[]
def create_test_data():
    """
        [1] Description: Create Test Data (Value of Image,Label)
        [2] Arguments: None
        [3] Return: testing_data (List)
    """
    testing_data=[]
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_data = cv2.imread('data\Test', 0)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        testing_data.append(np.array(img_data))
        labels.append(img)
    np.save('test_data.npy', testing_data)
    return testing_data


In [ ]:
'''if (os.path.exists('/content/train_data.npy')): # If you have already created the dataset:
    train_data =np.load(r'/content/train_data.npy', allow_pickle=True)
    #train_data = create_train_data()
else: # If dataset is not created:'''
train_data = create_train_data()

In [ ]:
'''if (os.path.exists(r'/content/test_data.npy')):
    test_data =np.load(r'/content/test_data.npy',allow_pickle=True)
else:'''
test_data = create_test_data()

In [ ]:
train = pd.DataFrame(train_data)

In [ ]:
train[1][1]

In [ ]:
train[0]=train[0]/255.

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(train[0],train[1], test_size=0.20,shuffle=True)
X_tr = np.array([i for i in X_tr]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_tr = [i for i in y_tr]
X_te = np.array([i for i in X_te]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_te = [i for i in y_te]

In [ ]:
class_names = ['Basketball', 'Yoga', 'Football', 'Swimming', 'Rowing',
               'Tennis']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train[0][i])
plt.show()

In [ ]:
tf.compat.v1.reset_default_graph()
conv_input = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')
conv1 = conv_2d(conv_input, 64, 2, activation='relu')
pool1 = max_pool_2d(conv1, 3)


fully_layer = fully_connected(pool1, 512, activation='relu')
fully_layer = dropout(fully_layer, 0.3)

cnn_layers = fully_connected(fully_layer, 6, activation='softmax')


cnn_layers = regression(cnn_layers, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
model = tflearn.DNN(cnn_layers, tensorboard_dir='log', tensorboard_verbose=3)
'''if (os.path.exists('model.tfl.meta')):
    model.load('./model.tfl')
else:'''
m=model.fit({'input': X_tr}, {'targets': y_tr}, 
               validation_set=({'input': X_te}, {'targets': y_te})
              , n_epoch=epochs,snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
model.save('model.tfl')

In [ ]:
FINAL_TEST_X = np.array([i for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
TEST_Y = model.predict(FINAL_TEST_X)

In [ ]:
TEST_Y

In [ ]:
FINAL_TEST_Y = []
for i in TEST_Y:
  max = -99999
  i=i[::-1]
  for d in range(0,len(i)):
    if i[d] > max:
        max = i[d]
        index = d
  FINAL_TEST_Y.append(index)

In [ ]:
final = pd.DataFrame()
final["image"] =  labels
final["label"] = FINAL_TEST_Y
final.join(pd.DataFrame(TEST_Y))

In [ ]:
final.to_csv('Final.csv',index=False)